# Skill estimation using Stan

In [24]:
import numpy as np
import pystan
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(66)
import pickle
import pandas as pd
from datetime import datetime

## Model that defined skill's distribution with more features

In [25]:
skill_model = """
data {
  int<lower=1> N;             // Total number of players
  int<lower=1> E;             // number of games
  real<lower=0> scale;        // scale value for probability computation
  int<lower=0,upper=1> win[E]; // PA wins vs PB
  int PA[E];                  // player info between each game
  int PB[E];                  // 
  real MA[E];                  // # Match
  real MB[E];                  //
}
parameters {
  vector [N] skill;           // skill values for each player
}

model{
  for (i in 1:N){ skill[i]~normal(0,3); }
  for (i in 1:E){
    win[i] ~ bernoulli_logit( (scale)*(skill[PA[i]]*exp(0.1/MA[i])-skill[PB[i]]*exp(0.1/MB[i])) );
  }   // win probability is a logit function of skill difference
}
"""

Now, compile the model.  

In [26]:
sm = pystan.StanModel(model_code = skill_model)

In [27]:
with open('skill_model_scores_time.pkl', 'wb') as f: pickle.dump(sm, f)

In [5]:
try:     # load it if already compiled
    sm = pickle.load(open('skill_model_scores_time.pkl', 'rb'))
except:  # ow, compile and save compiled model
    sm = pystan.StanModel(model_code = skill_model)
    with open('skill_model_scores_time.pkl', 'wb') as f: pickle.dump(sm, f)

## Processing data

In [28]:
def load_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    
    # Sparsifying parameters (discard some training examples):
    # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
    # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
    # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)

    wins = []
    playerA, playerB = [], []
    nplayers = len(playerid)
    nplays = np.zeros( (nplayers,nplayers) )
    scoresA = []
    scoresB = []
    scores = []
    
    matchA = []
    matchB = [] 
    
    train = pd.read_csv(dir+opt+'.csv', header = None)
    for i, csv in sorted(train.iterrows(), reverse = True, key=lambda date: datetime.strptime(date[1][0], "%m/%d/%Y")):

        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                
                nplays[a,b] += 1
                nplays[b,a] += 1
                
                matchA.append( nplays[a,:].sum() )
                matchB.append( nplays[b,:].sum() )      
                
                playerA.append(a+1)
                playerB.append(b+1)
                wins.append(1 if aw else 0) 
                sa, sb = csv[3].split('–')
                scores.append(abs(int(sa)-int(sb)))
                scoresA.append(int(sa))
                scoresB.append(int(sb))

    return playerid, nplayers,playerA,playerB, matchA, matchB, wins

In [29]:
playerid, nplayers,playerA,playerB, matchA, matchB, wins = load_data()

In [30]:
print('summary: ')
print('# players', nplayers)
print('# games', len(wins))
print('player A', playerA[:10])
print('player B', playerB[:10])
print('wins', wins[:10])
print('matchA', matchA[:10])
print('matchB', matchB[:10])

summary: 
# players 999
# games 4540
player A [208, 4, 2, 204, 204, 10, 153, 213, 13, 215]
player B [153, 2, 4, 422, 471, 215, 208, 13, 213, 10]
wins [1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
matchA [1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 2.0, 2.0]
matchB [1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 2.0]


We also need the observed data: number of players and games, which pairs played each game, and who won:

In [31]:
skill_data = {
    'N': nplayers,
    'E': len(wins),
    'scale': 0.4,
    'win':wins,
    'PA': playerA,
    'PB': playerB,
    'MA': matchA,
    'MB': matchB
}
# Player 1 & 3 played & P1 won; then again; then P2 & P3 (P2 wins), etc.

Now, we can perform MCMC on the model, and extract the samples:

In [32]:
fit = sm.sampling(data=skill_data, iter=2000, chains=4, n_jobs = 4, verbose = True)

In [33]:
samples = fit.extract()

If we just want the mean estimate for each player's skill level, just take the empirical average over the samples:

In [34]:
samples

OrderedDict([('skill',
              array([[ 1.65955832,  7.72685968,  4.92540836, ...,  0.49948073,
                      -1.45262942,  0.78167678],
                     [ 2.6608985 ,  7.3781883 ,  3.76790332, ..., -1.95266254,
                      -0.96340361, -4.76502423],
                     [-0.54136694,  8.37092847,  2.1250025 , ...,  0.02415816,
                      -0.15089311,  2.27656382],
                     ...,
                     [-0.1035078 ,  5.80660481,  4.71563206, ...,  2.73187595,
                      -1.59633627, -3.57027518],
                     [ 1.3835075 ,  8.83938511,  2.92576092, ..., -2.29024179,
                      -1.51235272, -2.91583245],
                     [-0.61120193,  5.05758366,  3.74456839, ..., -1.54431372,
                       0.51943468,  1.67043111]])),
             ('lp__',
              array([-2560.6665134 , -2568.7500978 , -2542.94635163, ...,
                     -2608.1498594 , -2593.53354302, -2563.23574929]))])

In [35]:
samples['skill'].shape # 2*100 iterations? 999 players

(4000, 999)

Remember to save the prediction!

In [36]:
with open('skill_hat_time.pkl', 'wb') as f: 
    pickle.dump(samples, f)

## Sample Model Evaluation

In [37]:
skill_hat = pickle.load(open('skill_hat_time.pkl', 'rb'))

In [38]:
skill_hat = skill_hat['skill']

In [39]:
def load_valid_data(playerid, dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='valid'):
        
  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    
    valid = pd.read_csv(dir+opt+'.csv', header = None)
    for i, csv in sorted(valid.iterrows(), reverse = True, key=lambda date: datetime.strptime(date[1][0], "%m/%d/%Y")):
    
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                nplays[a,b] += 1
                nplays[b,a] += 1
                nwins[a,b] += aw
                nwins[b,a] += bw
                games.append((a,b,aw,nplays[a,:].sum(),nplays[b,:].sum()))
    
    return nplayers, nplays, nwins, games

In [40]:
nplayers_val, nplays_val, nwins_val, games_val = load_valid_data(playerid)

In [41]:
print('summary: ', nplayers_val)
print(nplays_val.shape, nplays_val.sum())
print(nwins_val.shape, nwins_val.sum())
print('games', len(games_val))

summary:  999
(999, 999) 9362.0
(999, 999) 4687.0
games 4681


In [42]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = logit( skill_data['scale']*(skill[:,i]-skill[:,j]) ).mean()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [43]:
loss = prediction_loss(skill_hat, nplayers_val, nplays_val, nwins_val, games_val)

In [44]:
loss

(0.3952548128406488, 0.33345615327929257)

In [45]:
def logit(z): return 1./(1.+np.exp(-z))

def loss(skill, nplayers, games):
    
    loss = 0.
    binary_loss = 0.
    for game in games:
        a, b, w, ma, mb = game
        
        prob_hat = logit( skill_data['scale']*(skill[:,a]*np.exp(1/ma)-skill[:,b]*np.exp(1/mb)) ).mean()
        binary_loss += np.logical_xor(prob_hat >= 0.5, w)
    

    binary_loss /= len(games)
    
    return binary_loss


In [46]:
loss(skill_hat, nplayers_val, games_val)

0.35569322794274727